## Inbeddingen

In ons vorige voorbeeld werkten we met hoog-dimensionale bag-of-words vectoren met een lengte van `vocab_size`, en we waren expliciet bezig met het omzetten van laag-dimensionale positionele representatievectoren naar schaarse one-hot representaties. Deze one-hot representatie is niet geheugen-efficiënt en bovendien wordt elk woord onafhankelijk van de andere behandeld, d.w.z. one-hot gecodeerde vectoren drukken geen enkele semantische gelijkenis tussen woorden uit.

In deze eenheid gaan we verder met het verkennen van de **News AG** dataset. Om te beginnen, laten we de data laden en enkele definities uit het vorige notebook ophalen.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## Wat is embedding?

Het idee van **embedding** is om woorden te representeren door lagere-dimensionale, dense vectoren die op een bepaalde manier de semantische betekenis van een woord weerspiegelen. Later zullen we bespreken hoe we betekenisvolle woordembeddings kunnen bouwen, maar voor nu kunnen we embeddings zien als een manier om de dimensionaliteit van een woordvector te verlagen.

Een embedding-laag neemt een woord als invoer en produceert een uitvoervector met een gespecificeerde `embedding_size`. In zekere zin lijkt het erg op een `Linear`-laag, maar in plaats van een one-hot encoded vector te gebruiken, kan het een woordnummer als invoer nemen.

Door een embedding-laag als eerste laag in ons netwerk te gebruiken, kunnen we overschakelen van een bag-of-words naar een **embedding bag**-model, waarbij we eerst elk woord in onze tekst omzetten naar de bijbehorende embedding en vervolgens een aggregatiefunctie toepassen op al deze embeddings, zoals `sum`, `average` of `max`.

![Afbeelding die een embedding-classificator toont voor vijf sequentiewoorden.](../../../../../translated_images/nl/embedding-classifier-example.b77f021a7ee67eee.webp)

Ons classifier-neuraal netwerk zal beginnen met een embedding-laag, gevolgd door een aggregatielaag en een lineaire classifier bovenop:


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### Omgaan met variabele sequentiegrootte

Door deze architectuur moeten minibatches voor ons netwerk op een bepaalde manier worden gemaakt. In de vorige eenheid, bij het gebruik van bag-of-words, hadden alle BoW-tensors in een minibatch dezelfde grootte `vocab_size`, ongeacht de werkelijke lengte van onze tekstsequentie. Zodra we overstappen op woordembeddingen, krijgen we een variabel aantal woorden in elke tekstsample, en bij het combineren van die samples in minibatches moeten we enige padding toepassen.

Dit kan worden gedaan door dezelfde techniek te gebruiken waarbij een `collate_fn`-functie aan de datasource wordt geleverd:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### Classificatie van embeddings trainen

Nu we een juiste dataloader hebben gedefinieerd, kunnen we het model trainen met behulp van de trainingsfunctie die we in de vorige eenheid hebben gedefinieerd:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **Opmerking**: We trainen hier slechts voor 25k records (minder dan één volledige epoch) om tijd te besparen, maar je kunt doorgaan met trainen, een functie schrijven om voor meerdere epochs te trainen, en experimenteren met de leersnelheidsparameter om een hogere nauwkeurigheid te bereiken. Je zou een nauwkeurigheid van ongeveer 90% moeten kunnen behalen.


### EmbeddingBag-laag en representatie van variabele-lengte sequenties

In de vorige architectuur moesten we alle sequenties op dezelfde lengte opvullen om ze in een minibatch te passen. Dit is niet de meest efficiënte manier om sequenties met variabele lengte te representeren - een andere aanpak zou zijn om een **offset**-vector te gebruiken, die de offsets van alle sequenties in één grote vector bevat.

![Afbeelding die een offset-sequentierepresentatie toont](../../../../../translated_images/nl/offset-sequence-representation.eb73fcefb29b46ee.webp)

> **Note**: Op de afbeelding hierboven tonen we een sequentie van karakters, maar in ons voorbeeld werken we met sequenties van woorden. Het algemene principe van het representeren van sequenties met een offset-vector blijft echter hetzelfde.

Om met offset-representatie te werken, gebruiken we de [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html)-laag. Deze lijkt op `Embedding`, maar neemt een content-vector en een offset-vector als invoer, en bevat ook een averaging-laag, die `mean`, `sum` of `max` kan zijn.

Hier is een aangepaste netwerkarchitectuur die gebruikmaakt van `EmbeddingBag`:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

Om de dataset voor training voor te bereiden, moeten we een conversiefunctie bieden die de offsetvector zal voorbereiden:


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

Merk op dat, in tegenstelling tot alle eerdere voorbeelden, ons netwerk nu twee parameters accepteert: datavector en offsetvector, die van verschillende grootte zijn. Evenzo levert onze dataloader ons nu 3 waarden in plaats van 2: zowel tekst- als offsetvectoren worden als kenmerken geleverd. Daarom moeten we onze trainingsfunctie enigszins aanpassen om hiermee rekening te houden:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## Semantische Embeddings: Word2Vec

In ons vorige voorbeeld leerde de embeddinglaag van het model om woorden naar vectorrepresentaties te mappen, maar deze representatie had niet veel semantische betekenis. Het zou fijn zijn om een vectorrepresentatie te leren waarbij vergelijkbare woorden of synoniemen overeenkomen met vectoren die dicht bij elkaar liggen in termen van een bepaalde vectorafstand (bijv. euclidische afstand).

Om dat te bereiken, moeten we ons embeddingmodel op een grote verzameling tekst op een specifieke manier voortrainen. Een van de eerste methoden om semantische embeddings te trainen wordt [Word2Vec](https://en.wikipedia.org/wiki/Word2vec) genoemd. Het is gebaseerd op twee hoofdarchitecturen die worden gebruikt om een gedistribueerde representatie van woorden te produceren:

 - **Continuous bag-of-words** (CBoW) — in deze architectuur trainen we het model om een woord te voorspellen op basis van de omliggende context. Gegeven de n-gram $(W_{-2},W_{-1},W_0,W_1,W_2)$, is het doel van het model om $W_0$ te voorspellen op basis van $(W_{-2},W_{-1},W_1,W_2)$.
 - **Continuous skip-gram** is het tegenovergestelde van CBoW. Het model gebruikt het omliggende venster van contextwoorden om het huidige woord te voorspellen.

CBoW is sneller, terwijl skip-gram langzamer is, maar beter presteert bij het representeren van zeldzame woorden.

![Afbeelding die zowel CBoW- als Skip-Gram-algoritmen toont om woorden naar vectoren te converteren.](../../../../../translated_images/nl/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Om te experimenteren met Word2Vec-embedding die is voorgetraind op de Google News dataset, kunnen we de **gensim**-bibliotheek gebruiken. Hieronder vinden we de woorden die het meest lijken op 'neural'.

> **Opmerking:** Wanneer je voor het eerst woordvectoren aanmaakt, kan het downloaden ervan enige tijd duren!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


We kunnen ook vector-embeddings berekenen vanuit het woord, om te gebruiken bij het trainen van een classificatiemodel (we tonen alleen de eerste 20 componenten van de vector voor de duidelijkheid):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Het geweldige aan semantische embeddings is dat je de vectorcodering kunt manipuleren om de semantiek te veranderen. Bijvoorbeeld, we kunnen vragen om een woord te vinden waarvan de vectorrepresentatie zo dicht mogelijk bij de woorden *koning* en *vrouw* ligt, en zo ver mogelijk van het woord *man*:


In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Zowel CBoW als Skip-Grams zijn "voorspellende" embeddings, omdat ze alleen lokale contexten in aanmerking nemen. Word2Vec maakt geen gebruik van globale context.

**FastText** bouwt voort op Word2Vec door vectorrepresentaties te leren voor elk woord en de karakter n-grams die binnen elk woord voorkomen. De waarden van de representaties worden vervolgens gemiddeld tot één vector bij elke trainingsstap. Hoewel dit veel extra berekeningen toevoegt aan de pre-training, stelt het woordembeddings in staat om subwoordinformatie te coderen.

Een andere methode, **GloVe**, maakt gebruik van het idee van een co-occurrentie matrix en gebruikt neurale methoden om de co-occurrentie matrix te ontleden in meer expressieve en niet-lineaire woordvectoren.

Je kunt met het voorbeeld spelen door de embeddings te veranderen naar FastText en GloVe, aangezien gensim verschillende modellen voor woordembeddings ondersteunt.


## Gebruik van Voorgetrainde Embeddings in PyTorch

We kunnen het bovenstaande voorbeeld aanpassen om de matrix in onze embedding-laag vooraf te vullen met semantische embeddings, zoals Word2Vec. We moeten er rekening mee houden dat de woordenschat van de voorgetrainde embedding en ons tekstcorpus waarschijnlijk niet overeenkomen, dus we zullen de gewichten voor de ontbrekende woorden initialiseren met willekeurige waarden:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


Laten we nu ons model trainen. Merk op dat de tijd die het kost om het model te trainen aanzienlijk groter is dan in het vorige voorbeeld, vanwege de grotere grootte van de inbeddingslaag en daardoor een veel hoger aantal parameters. Ook kunnen we hierdoor ons model op meer voorbeelden moeten trainen als we overfitting willen vermijden.


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

In ons geval zien we geen enorme toename in nauwkeurigheid, wat waarschijnlijk te maken heeft met behoorlijk verschillende woordenschatten.  
Om het probleem van verschillende woordenschatten te overwinnen, kunnen we een van de volgende oplossingen gebruiken:  
* Het word2vec-model opnieuw trainen op onze woordenschat  
* Onze dataset laden met de woordenschat van het vooraf getrainde word2vec-model. De woordenschat die wordt gebruikt om de dataset te laden, kan tijdens het laden worden gespecificeerd.  

De laatste aanpak lijkt eenvoudiger, vooral omdat het PyTorch `torchtext` framework ingebouwde ondersteuning voor embeddings bevat. We kunnen bijvoorbeeld een GloVe-gebaseerde woordenschat instantiëren op de volgende manier:  


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


Het geladen vocabulaire heeft de volgende basisbewerkingen:
* De `vocab.stoi`-woordenlijst stelt ons in staat om een woord om te zetten naar zijn index in de woordenlijst.
* `vocab.itos` doet het tegenovergestelde - het zet een nummer om naar een woord.
* `vocab.vectors` is de array van embedding-vectoren, dus om de embedding van een woord `s` te krijgen, moeten we `vocab.vectors[vocab.stoi[s]]` gebruiken.

Hier is een voorbeeld van het manipuleren van embeddings om de vergelijking **kind-man+woman = queen** te demonstreren (ik moest de coëfficiënt een beetje aanpassen om het te laten werken):


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

Om de classifier te trainen met behulp van die embeddings, moeten we eerst onze dataset coderen met behulp van de GloVe-woordenschat:


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

Zoals we hierboven hebben gezien, worden alle vector-embeddings opgeslagen in de `vocab.vectors` matrix. Het maakt het super eenvoudig om die gewichten te laden in de gewichten van de embedding-laag door simpelweg te kopiëren:


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

Een van de redenen waarom we geen significante toename in nauwkeurigheid zien, is het feit dat sommige woorden uit onze dataset ontbreken in de voorgetrainde GloVe-woordenschat en daardoor in wezen worden genegeerd. Om dit te verhelpen, kunnen we onze eigen embeddings trainen op onze dataset.


## Contextuele Embeddings

Een belangrijke beperking van traditionele vooraf getrainde embedding-representaties zoals Word2Vec is het probleem van woordbetekenis-onderscheiding. Hoewel vooraf getrainde embeddings een deel van de betekenis van woorden in context kunnen vastleggen, wordt elke mogelijke betekenis van een woord in dezelfde embedding gecodeerd. Dit kan problemen veroorzaken in downstream-modellen, aangezien veel woorden, zoals het woord 'play', verschillende betekenissen hebben afhankelijk van de context waarin ze worden gebruikt.

Bijvoorbeeld, het woord 'play' heeft in deze twee verschillende zinnen een heel andere betekenis:
- Ik ging naar een **toneelstuk** in het theater.
- John wil **spelen** met zijn vrienden.

De hierboven genoemde vooraf getrainde embeddings vertegenwoordigen beide betekenissen van het woord 'play' in dezelfde embedding. Om deze beperking te overwinnen, moeten we embeddings bouwen op basis van het **taalmodel**, dat is getraind op een grote hoeveelheid tekst en *begrijpt* hoe woorden in verschillende contexten kunnen worden gebruikt. Het bespreken van contextuele embeddings valt buiten de scope van deze tutorial, maar we zullen hierop terugkomen wanneer we taalmodellen bespreken in de volgende eenheid.



---

**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsservice [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we ons best doen voor nauwkeurigheid, dient u zich ervan bewust te zijn dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in zijn oorspronkelijke taal moet worden beschouwd als de gezaghebbende bron. Voor cruciale informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor eventuele misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
